In [12]:
%run efx_charity
run()

loaded efx algorithm


number of players:  3
number of item types:  2
number of items of each type (comma-separated):  40,40
valuation for player 0 (comma-separated):  2,1
valuation for player 1 (comma-separated):  1,5
valuation for player 2 (comma-separated):  6,100


APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
COULD NOT APPLY U0


NameError: name 'new_assignment' is not defined

In [ ]:
assignm

In [2]:
run_and_check(draw=True)

KeyboardInterrupt: Interrupted by user